# Mail Classification Model

## Importing Libraries

In [13]:
import os
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

### Load Dataset from Files

In [ ]:
# The dataset is in the folder DATA_FOLDER
DATA_FOLDER = 'new_data'
texts = []
labels = []
folders = ['ham', 'spam', 'social', 'promotion']
for folder in folders:
    path = f'{DATA_FOLDER}/{folder}'
    files = os.listdir(path)
    print('#Files: ', len(files))
    
    labels += [folder]*len(files)
    
    for file in tqdm(files):
        text = open(f'{path}/{file}', encoding='utf-8').read()
        texts.append(text)

In [ ]:
df = pd.DataFrame(data=[texts, labels]).T
df.columns = ['text', 'labels']

### Save Dataset

In [ ]:
pickling_on = open("./dataset.pkl","wb")
pickle.dump(df,pickling_on)
pickling_on.close()

### Load Saved Dataset

In [3]:
pickling_on = open("./dataset.pkl","rb")
df = pickle.load(pickling_on)
pickling_on.close()

### TFIDF Vectorizer Fit and Transform

In [4]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,3), max_features=200)
x = vectorizer.fit_transform(df['text'])

### Save Vectorizer

In [62]:
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))

### Load Vectorizer

In [ ]:
vectorizer = pickle.load(open('vectorizer.pkl', 'wb'))

### Train Test Split of Dataset

In [5]:
X_train, X_test, y_train, y_test = train_test_split(x, df.labels, test_size=0.10, random_state=42, shuffle = True, stratify=df.labels)

# SVM MODEL

## Create Model and Train

In [ ]:
model = LinearSVC(C=10, class_weight='balanced', dual=True, fit_intercept=True,
                intercept_scaling=1, loss='squared_hinge', max_iter=1000,
                multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=True)

model.fit(X_train, y_train)

### Saving SVM Model

In [ ]:
pickling_on = open("./model.pkl","wb")
pickle.dump(model,pickling_on)
pickling_on.close()

### Loading Saved Model

In [6]:
pickling_on = open("./model.pkl","rb")
model = pickle.load(pickling_on)
pickling_on.close()

/home/kaush/.local/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LinearSVC from version 0.20.3 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### Test Data Prediction

In [7]:
y_pred = model.predict(X_test)

## Model's Accuracy Score

In [8]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy : {0:.2f} %".format(accuracy*100))

Accuracy : 94.12 %


## Model's Confusion Matrix

In [9]:
print(confusion_matrix(y_test, y_pred))

[[1521    1    1  132]
 [   0  164    1    2]
 [   1    2  129    2]
 [  72    1    1 1642]]


## Model's Classification Report

In [10]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         ham       0.95      0.92      0.94      1655
   promotion       0.98      0.98      0.98       167
      social       0.98      0.96      0.97       134
        spam       0.92      0.96      0.94      1716

    accuracy                           0.94      3672
   macro avg       0.96      0.96      0.96      3672
weighted avg       0.94      0.94      0.94      3672



# LSTM MODEL

## Build the model

In [50]:
clear_session()
model = Sequential()
model.add(LSTM(32, input_shape=(200, 1)))
model.add(Dense(4, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                4352      
_________________________________________________________________
dense (Dense)                (None, 4)                 132       
Total params: 4,484
Trainable params: 4,484
Non-trainable params: 0
_________________________________________________________________
None


## If the model has been trained already, load the model
### Run this only after the model has been trained and saved

In [49]:
model = load_model('./model.h5')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 16)                1152      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,220
Trainable params: 1,220
Non-trainable params: 0
_________________________________________________________________
None


## We can use the same X_train and X_test data for the LSTM model. By using the same input data, we can compare the models quite well.

**But, we need to encode the Labels. **

In [18]:
encoder = LabelEncoder()
y_train_lstm = encoder.fit_transform(y_train)
y_test_lstm = encoder.fit_transform(y_test)

## Save the encoder

In [19]:
pickle.dump(encoder, open('./encoder.pkl', 'wb'))

## Load the encoder

In [20]:
encoder = pickle.load(open('./encoder.pkl', 'rb'))

## For LSTM, we need to reshape the data

In [38]:
X_train_lstm = X_train.toarray().reshape(-1, 200, 1)
X_test_lstm = X_test.toarray().reshape(-1, 200, 1)
print(X_train_lstm.shape, X_test_lstm.shape)

(33044, 200, 1) (3672, 200, 1)


## Callbacks for saving the model and stopping training if validation accuracy does not improve

In [51]:
callbacks = [
    EarlyStopping(patience=10),
    ModelCheckpoint('./model.h5', period=5)
]

## Compile the model and fit the model on the data

In [52]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.fit(X_train_lstm, y_train_lstm, batch_size=512, epochs=250, validation_split=0.1, callbacks=callbacks)

Train on 29739 samples, validate on 3305 samples
Epoch 1/250
29739/29739 [==============================] - 14s 482us/sample - loss: 1.1495 - val_loss: 0.9745
Epoch 2/250
29739/29739 [==============================] - 14s 465us/sample - loss: 0.9782 - val_loss: 0.9708
Epoch 3/250
29739/29739 [==============================] - 14s 484us/sample - loss: 0.9768 - val_loss: 0.9703
Epoch 4/250
29739/29739 [==============================] - 14s 479us/sample - loss: 0.9761 - val_loss: 0.9694
Epoch 5/250
29739/29739 [==============================] - 14s 470us/sample - loss: 0.9744 - val_loss: 0.9734
Epoch 6/250
29739/29739 [==============================] - 15s 494us/sample - loss: 0.9720 - val_loss: 0.9626
Epoch 7/250
29739/29739 [==============================] - 14s 487us/sample - loss: 0.9527 - val_loss: 0.9266
Epoch 8/250
29739/29739 [==============================] - 14s 484us/sample - loss: 0.9099 - val_loss: 0.9111
Epoch 9/250
29739/29739 [==============================] - 15s 504us/sa

## Predict the labels

In [53]:
preds = model.predict_classes(X_test_lstm)

## Confusion Matrix

In [54]:
print(confusion_matrix(y_test_lstm, preds))

[[1185    7    8  455]
 [   5  128   16   18]
 [   7   11   92   24]
 [  98    3   11 1604]]


## Accuracy

In [59]:
accuracy = accuracy_score(y_test_lstm, preds)
print("Accuracy : {0:.2f} %".format(accuracy*100))

Accuracy : 81.94 %


## Classification Report

In [60]:
print(classification_report(y_test_lstm, preds))

              precision    recall  f1-score   support

           0       0.92      0.72      0.80      1655
           1       0.86      0.77      0.81       167
           2       0.72      0.69      0.70       134
           3       0.76      0.93      0.84      1716

    accuracy                           0.82      3672
   macro avg       0.82      0.78      0.79      3672
weighted avg       0.83      0.82      0.82      3672

